<a href="https://colab.research.google.com/github/1kruidenp/PRA3024/blob/master/Week_3_exercies1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The code below represents exercise 1 and 2, and computes the fischer matrix and the standard deviation of observables. The code can be modified so that the user can change to any waveform.

In [0]:
#here the necessary packages are installed
import scipy.integrate as integrate
import scipy.special as special
import scipy.misc as derivative
import math
import numpy as np



In [79]:


#All the constants are defined below

f0=150 #Hz
fs=40 #Hz #Minimum freq #Also the lower bound for the integral
m1=20**30 
m2=10**30
S0=9*10**-46 #Hz^-1
pi=math.pi
sqrt=math.sqrt
c=3*10**8
G=6.674*10**-11
SNR=10
#The 4 observables are below, they can be modified as needed
M=m1+m2 #
eta = (m1*m2)/(M**2)
tc=1
phi=1

#Below are the corrections used for the psi function, more can be added or less can be used, but make sure to modifiy this in the sum equation below
a0=1
a1=0
a2=(20/9)*((743/336)+(11/4)*eta)
a3=-16*pi
a4=10*((3058673/1016064)+(5429/1008)*eta+(617/144)*eta**2)

v=(pi*M*f)**(1/3)
#Don't forget to add correction
SUM=(a0*v**0)+(a1*v**1)+(a2*v**2)+(a3*v**3)+(a4*v**4)

f=215 

#Here are the variables which will be used to find the partial derivatives 
variables = [tc,phi,eta,M]
#These are also changed to have a different object type
observables= ["tc","phi","eta","M"]

#Here the sensitivity curve is created as a function
def Sen(f):
  x=f/fs  
  return S0*(((4.49*x)**-56)+0.16*(x**-4.52)+0.52+0.32*(x**2))
print(Sen(f))


8.788571953481794e-45


In [80]:
#A is created here which is needed later in the fourier transform
def A(f,fs):
  
  fh=(((6**(3/2))*pi*M)**(-1))*(c**3)*(1/G) #Upper bound for the integral. Missing G's and c's. wE HAVE 1/kg so c^3/G= c*m^3/s^3 * kg*s^2/m^3= kg/s which cancels to 1/s
  ins=lambda x:((f**(-7/6))/Sen(f))
  I=integrate.quad(ins, fs, fh)
  return sqrt((SNR)/(4*I[0]))


print (A(f, fs))
  

5.1608899488929084e-23


In [81]:
#Here we create the psi function


def psi(f):
  return (2*pi*f*tc)-phi-(pi/4)+(3/(128*eta*(v**5)))*SUM
print(psi(f))



1349.0994428802526


In [82]:
#Now we will make a function for the fourier transform of the signal.
#Two values are found here, one being real and the other imaginary
e=math.e

def h_r(f):
  H_R=A(f, fs)*(f**(-7/6))*e**(1j*psi(f))
  return H_R.real
def h_c(f):
  H_R=A(f, fs)*(f**(-7/6))*e**(1j*psi(f))
  return H_R.imag





print(h_r(f))
print (h_c(f))

-2.0885323233589236e-26
-9.58226530006541e-26


In [0]:
#Here a function is made to make the derivatives of the function
def d_h_real(i,inc):
  return (h_r(variables[i]+inc) - h_r(variables[i])) / (inc)
def d_h_compl(j,inc):
  return (h_c(variables[j]+inc) - h_c(variables[j])) / (inc)

In [0]:
def create_fm(matrix, increment):
  #This part here creates the range of frequencies to integrate from
  frequencies = np.zeros(shape=(len(variables)*len(variables)))
  start_freq = fs
  f_steps = int(  (f0-fs)  /  (len(variables)*len(variables))    )
  for s in range(len(variables)*len(variables)):
    frequencies[s] = start_freq
    start_freq += f_steps
  
  f_index = 0
  intg = 0
  #print(frequencies)
  for i in range(len(variables)):
    for j in range(len(variables)):
      if i==j:
        numerator = 4 * float(d_h_real(i, increment) * d_h_compl(j, increment))
        intg += (numerator/Sen(frequencies[f_index]))*increment
        matrix[i][j] = intg
      f_index += 1
  
  return matrix


In [85]:
#here we create fischer matrix.
fm = np.zeros( shape=(len(variables),len(variables)) )

create_fm(fm, 0.01)
print(fm)

[[ 8.24714713e+50  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  1.36988970e+51  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -3.96329956e+85  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -3.96329956e+85]]


In [0]:
#The standard deviation of the observables are calculated here
def compute_stds(matrix):
  inverse = np.linalg.inv(matrix)
  diag_elements = []
  for i in range(len(variables)):
    for j in range(len(variables)):
      if (i==j):
        diag_elements.append(inverse[i][j])
  for k in range(len(diag_elements)):
    diag_elements[k] = np.sqrt(diag_elements[k])

  for l in range(len(diag_elements)):
    print("The standard deviation for {} is {}".format(observables[l],diag_elements[l]))

In [89]:
compute_stds(fm)


The standard deviation for tc is 3.482155241074072e-26
The standard deviation for phi is 2.7018248972420034e-26
The standard deviation for eta is nan
The standard deviation for M is nan


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in sqrt
  if __name__ == '__main__':
